In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.graph_objects as go

# ML
from regression import PCovR, KPCovR, SparseKPCovR
from regression import LR, KRR
from kernels import build_kernel, linear_kernel, gaussian_kernel
from kernels import center_kernel, center_kernel_fast
from kernels import center_kernel_oos, center_kernel_oos_fast
from soap import compute_soap_density, reshape_soaps
from soap import rrw_neighbors, make_tuples

from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LogisticRegression

# Atoms
from ase.io import read
from ase.neighborlist import neighbor_list

# Utilities
import h5py
import json
import itertools
from copy import deepcopy
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

In /home/helfrech/.config/matplotlib/stylelib/cosmo.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /home/helfrech/.config/matplotlib/stylelib/cosmoLarge.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
/home/helfrech/ENVIRONMENTS/ZEOLITES/lib/python3.6/_collections_abc.py:841: MatplotlibDeprecationWarning:


The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.



In [3]:
sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/kernel-tutorials')
sys.path.append('/scratch/helfrech/Sync/GDrive/Projects/KPCovR/KernelPCovR/analysis/scripts')
# from utilities.sklearn_covr.kpcovr import KernelPCovR as KPCovR2
# from utilities.sklearn_covr.pcovr import PCovR as PCovR2
from helpers import l_regr, l_kpcovr, l_proj

# Functions

In [ ]:
def load_soaps(deem_file, iza_file, 
               idxs_deem_train, idxs_deem_test, 
               idxs_iza_train, idxs_iza_test,
               idxs_deem_delete=[], idxs_iza_delete=[]):
    
    # Load SOAPs
    soaps_deem = load_structures_from_hdf5(deem_file, datasets=None, concatenate=False)
    for i in sorted(idxs_deem_delete, reverse=True):
        soaps_deem.pop(i)
    
    soaps_iza = load_structures_from_hdf5(iza_file, datasets=None, concatenate=False)
    for i in sorted(idxs_iza_delete, reverse=True):
        soaps_iza.pop(i)
        
    # Build the train and test sets
    deem_train = [soaps_deem[i] for i in idxs_deem_train]
    deem_test = [soaps_deem[i] for i in idxs_deem_test]
    iza_train = [soaps_iza[i] for i in idxs_iza_train]
    iza_test = [soap_iza[i] for i in idxs_iza_test]
    
    soaps_train = iza_train + deem_train
    soaps_test = iza_test + deem_test
    
    return soaps_train, soaps_test

def preprocess_soaps(soaps_train, soaps_test):
        
    # Can also do other scaling/centering here -- 
    # this is mostly just to get the SOAPs to a 'usable' magnitude
    soaps_scale = np.std(soaps_train)
    soaps_train_scaled = soaps_train / soaps_scale
    soaps_test_scaled = soaps_test / soaps_scale
    
    return soaps_train, soaps_test

def load_decision_functions(deem_file, iza_file,
                            idxs_deem_train, idxs_deem_test,
                            idxs_iza_train, idxs_iza_test):
    
    deem_data = np.loadtxt(deem_file)
    iza_data = np.loadtxt(iza_file)
        
    deem_data_train = deem_data[idxs_deem_train]
    deem_data_test = deem_data[idxs_deem_test]
    
    iza_data_train = iza_data[idxs_iza_train]
    iza_data_test = iza_data[idxs_iza_test]
    
    train_data = np.concatenate((iza_data_train, deem_data_train))
    test_data = np.concatenate((iza_data_test, deem_data_test))
    
    return train_data, test_data

def load_kernels(kernel_file):
    
    # Load the kernels
    f = h5py.File(kernel_file, 'r')

    K_train = f['K_train'][:]
    K_test = f['K_test'][:]
    K_test_test = f['K_test_test'][:]

    f.close()
    
    return K_train, K_test, K_test_test
    
def compute_kernels(soaps_train, soaps_test, kernel_file=None, **kwargs):
    

    # Build kernel between all DEEM and all IZA
    K_train = build_kernel(soaps_train, soaps_train, **kwargs)
    K_test = build_kernel(soaps_test, soaps_train, **kwargs)
    K_test_test = build_kernel(soaps_test, soaps_test, **kwargs)
        
    if kernel_file is not None:
        
        # Save kernels for later
        g = h5py.File(kernel_file, 'w')

        g.create_dataset('K_train', data=K_train)
        g.create_dataset('K_test', data=K_test)
        g.create_dataset('K_test_test', data=K_test_test)

        for k, v in kernel_parameters.items():
            g.attrs[k] = v

        g.close()
    
    return K_train, K_test, K_test_test
    
def preprocess_kernels(K_train, K_test=[], K_test_test=[]):
    
    K_test_test = [center_kernel(k, K_ref=K_train) for k in K_test_test]
    K_test = [center_kernel(k, K_ref=K_train) for k in K_test]
    K_train = center_kernel(K_train)
    
    K_scale = np.trace(K_train) / K_train.shape[0]
    
    K_test_test = [k / K_scale for k in K_test_test]
    K_test = [k / K_scale for k in K_test]
    K_train /= K_scale
    
    return K_train, K_test, K_test_test

def do_svc(train_data, test_data, train_classes, test_classes,
           svc_type='linear', outputs=['decision_functions', 'predictions', 'scores'], **kwargs):
    
    if svc_type == 'kernel':
        svc = SVC(**kwargs)
        
    elif svc_type == 'linear':
        svc = LinearSVC(**kwargs)
        
    else:
        print("Error: invalid svc_type; valid choices are 'kernel' and 'linear'")
        return

    svc.fit(train_data, train_classes)
    
    output_list = []
    
    # Structure in this way to return in the same order as given in the outputs list
    for out in outputs:
        if out == 'decision_functions':
            df_train = svc.decision_function(train_data)
            df_test = svc.decision_function(test_data)
            output_list.extend((df_train, df_test))
            
        elif out == 'predictions':
            predicted_train_classes = svc.predict(train_data)
            predicted_test_classes = svc.predict(test_data)
            output_list.extend((predicted_train_classes, predicted_test_classes))
        
            print(classification_report(test_classes, predicted_test_classes))
            print(confusion_matrix(test_classes, predicted_test_classes))
            
        elif out == 'scores':
            train_score = svc.score(train_data, train_classes)
            test_score = svc.score(test_data, test_classes)
            output_list.extend((train_scores, test_scores))

            print(train_score)
            print(test_score)
            
    return output_list
           
def regression_check(train_data, test_data,
                     train_target, test_target,
                     regression_type='linear'):
    
    if regression_type == 'linear':
        regression_func = LR
        
    elif regression_type == 'kernel':
        regression_func = KRR
        
    else:
        print("Error: invalid regression type; use 'linear' or 'kernel'")
        return

    # Test KRR on decision functions
    # NOTE: KRR can't predict the test set
    # decision function very well -- why? <-- TODO: is this only for LinearSVC or also SVC?
    
#     regressor = KernelRidge(alpha=1.0E-12, kernel='precomputed')
#     regressor.fit(train_data, train_target)
#     predicted_train_target = regressor.predict(train_data)
#     predicted_test_target = regressor.predict(test_data)

    regressor = regression_func(regularization=1.0E-12)
    regressor.fit(train_data, train_target)
    predicted_train_target = regressor.transform(train_data)
    predicted_test_target = regressor.transform(test_data)
    
    print(np.mean(np.abs(predicted_train_target - train_target), axis=0))
    print(np.mean(np.abs(predicted_test_target - test_target), axis=0))

def preprocess_decision_functions(df_train, df_test):
    df_center = np.mean(df_train, axis=0)
    
    df_train -= df_center
    df_test -= df_center
    
    if df_train.ndim == 1:
        df_scale = np.linalg.norm(df_train) / np.sqrt(df_train.size)
    else:
        df_scale = np.linalg.norm(df_train, axis=0) / np.sqrt(df_train.shape[0] / df_train.shape[1])
    
    df_train /= df_scale
    df_test /= df_scale
    
    return df_train, df_test, df_center, df_scale

def postprocess_decision_functions(df_train, df_test, df_center, df_scale):
    
    # Rescale to raw decision function
    dfp_train = dfp_train * df_scale + df_center
    dfp_test = dfp_test * df_scale + df_center

    # Predict classes based on KPCovRized decision functions
    predicted_cantons_train = df_to_class(dfp_train, df_type, n_classes)
    predicted_cantons_test = df_to_class(dfp_test, df_type, n_classes)
    
def split_and_save(train_data, test_data, 
                   train_idxs, test_idxs, 
                   train_slice, test_slice, 
                   hdf5_attrs=None, 
                   output, output_format='%f'):
    
    # Save KPCovR class predictions
    n_samples = len(train_data) + len(test_data)
    data = np.zeros((n_samples, train_data.shape[1]))
    data[train_idxs] = train_data[train_slice]
    data[test_idxs] = test_data[test_slice]
    
    if hdf5_attrs is not None:
        n_digits = len(str(n_samples - 1))
        g = h5py.File(output, 'w')
        for ddx, d in enumerate(data):
            g.create_dataset(str(ddx).zfill(n_digits), data=d)

        for k, v in hdf5_attrs.items():
            g.attrs[k] = v

        g.close()
    else:
        np.savetxt(output, data, fmt=output_format)

def do_covr(train_data, test_data, 
            train_targets, test_targets,
            covr_type='linear', **covr_parameters):
    
    if covr_type == 'linear':
        covr_func = PCovR
    elif covr_type == 'kernel':
        covr_func = KPCovR
    else:
        print("Error: invalid CovR type; use 'linear' or 'gaussian'")
    
    covr = covr_func(**covr_parameters)
    
    covr.fit(train_data, train_targets)
    
    T_train = covr.transform_K(train_data)
    predicted_train_target = covr.transform_Y(train_data)
    T_test = covr.transform_K(test_data)
    predicted_test_target = covr.transform_Y(test_data)
    
    predicted_train_target = np.squeeze(dfp_train) # TODO: move the squeezing to the KPCovR function
    predicted_test_target = np.squeeze(dfp_test)
    
    return T_train, T_test, predicted_train_target, predicted_test_target

# Load train and test splits

In [6]:
# Load SOAP cutoffs
with open('../Processed_Data/soap_hyperparameters.json', 'r') as f:
    soap_hyperparameters = json.load(f)
    
cutoffs = soap_hyperparameters['interaction_cutoff']

In [5]:
# Load train and test set indices for Deem
idxs_deem = np.loadtxt('../Processed_Data/DEEM_10k/train.idxs', dtype=int)

# Total number of structures
n_deem = idxs_deem.size + np.loadtxt('../Processed_Data/DEEM_10k/test.idxs', dtype=int).size

In [ ]:
# Select DEEM sample for training/testing (already shuffled)
n_deem_train= 5000
n_deem_test = 2750
idxs_deem_train = idxs_deem[0:n_deem_train]
idxs_deem_test = idxs_deem[n_deem_train:n_deem_train+n_deem_test]

In [10]:
# Make dummy DEEM cantons
cantons_deem = np.ones(n_deem, dtype=int) * 4

In [11]:
# Make dummy DEEM cantons
cantons_deem = np.ones(n_deem, dtype=int) * 2

In [ ]:
# Load IZA cantons
cantons_iza = np.loadtxt('../Raw_Data/GULP/IZA_226/cantons.txt', usecols=1, dtype=int)
RWY = np.nonzero(cantons_iza == 4)[0][0]

In [7]:
cantons_iza = np.delete(cantons_iza, RWY)
n_iza = len(cantons_iza)

In [8]:
cantons_iza = np.ones(n_iza, dtype=int)

In [9]:
# Select IZA sample
idxs_iza_train = np.arange(0, n_iza)
idxs_iza_test = np.arange(0, n_iza)

In [17]:
# Build set of "master" canton labels
cantons_train = np.concatenate((cantons_iza[idxs_iza_train], cantons_deem[idxs_deem_train]))
cantons_test = np.concatenate((cantons_iza[idxs_iza_test], cantons_deem[idxs_deem_test]))
n_classes = np.amax(cantons_train)

In [ ]:
# TODO: assign slices for saving

# Model setup

In [ ]:
kernel_model_dir = '../Processed_Data/Models/Kernel_Models'
linear_model_dir = '../Processed_Data/Models/Linear_Models'
deem_dir = '../Processed_Data/DEEM_10k/Data'
iza_dir = '../Processed_Data/IZA_226onDEEM_10k/Data'

In [ ]:
# Global model parameters
# TODO: or use .get_params()?
svc_kwargs = dict(linear=dict(penalty='l2',
                              loss='squared_hinge',
                              dual=False,
                              multi_class='ovr',
                              class_weight=None,
                              fit_intercept=True,
                              intercept_scaling=1.0,
                              tol=1.0E-3),
                  kernel=dict(kernel='precomputed',
                              decision_function_shape='ovr',
                              class_weight=None,
                              break_ties=False,
                              tol=1.0E-3))

# TODO: how to best assign n_components?
n_components = 2
covr_kwargs = dict(linear=dict(n_components=n_components),
                 kernel=dict(n_components=n_components))

C = np.logspace(-5, 5, 11)
alphas = np.linspace(0.0, 1.0, 11)
regularizations = np.logspace(-12, -1, 12)

## Optimization

In [1]:
n_subsets = 5

In [ ]:
# Split DEEM into subsets for optimization and validation
idxs_deem_train_kernel = np.arange(idxs_iza_train.size, idxs_iza_train.size+idxs_deem_train.size)
np.random.shuffle(idxs_deem_train_kernel)
idxs_deem_train_kernel = np.split(idxs_deem_train_kernel, n_subsets)

idxs_deem_test_kernel = np.arange(idxs_iza_test.size, idxs_iza_test.size+idxs_deem_test.size)
np.random.shuffle(idxs_deem_test_kernel)
idxs_deem_test_kernel = np.split(idxs_deem_test_kernel, n_subsets)

In [ ]:
# Split IZA into subsets for optimization and validation
idxs_iza = np.arange(0, n_iza)
n_iza_train_kernel = int(n_iza / 2)
n_iza_test_kernel = n_iza - n_iza_train_kernel

idxs_iza_train_kernel = []
idxs_iza_test_kernel = []
for n in range(0, n_subsets):
    np.random.shuffle(idxs_iza)
    idxs_iza_train_kernel.append(idxs_iza[0:n_iza_train_kernel])
    idxs_iza_test_kernel.append(idxs_iza[n_iza_train_kernel:])

In [ ]:
# Concatenate the DEEM and IZA indices
idxs_train_kernel = [np.concatenate((iza, deem)) for iza, deem in 
                     zip(idxs_iza_train_kernel, idxs_deem_train_kernel)]
idxs_test_kernel = [np.concatenate((iza, deem)) for iza, deem in
                    zip(idxs_iza_test_kernel, idxs_deem_test_kernel)]

## Model evaluation

In [19]:
# IZA indices for training and testing
idxs_iza = np.arange(0, n_iza)
n_iza_train = int(n_iza / 2)
n_iza_test = n_iza - n_iza_train

np.random.shuffle(idxs_iza)
idxs_iza_train_all = idxs_iza[0:n_iza_train]
idxs_iza_test_all = idxs_iza[n_iza_train:]

In [20]:
# DEEM indices for training and testing
idxs_deem_train_all = np.concatenate(idxs_deem_train_kernel)
idxs_deem_test_all = np.concatenate(idxs_deem_test_kernel)

In [21]:
# Concatenate the DEEM and IZA indices
idxs_train_all = np.concatenate((idxs_iza_train_all, idxs_deem_train_all))
idxs_test_all = np.concatenate((idxs_iza_test_all, idxs_deem_test_all))

# Build kernels

In [ ]:
# Load the kernels or compute if they don't exist
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        # TODO: MOVE KERNELS BEFORE RUNNING!
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_model_dir}/{kernel_name}/{cutoff}/structure_kernels_optimization.hdf5'
        kernel_parameter_file = f'{kernel_model_dir}/{kernel_name}/{cutoff}/volumes_mae_parameters.json'

        if not os.path.exists(kernel_file):
            
            # SOAP files (atomwise, FPS'ed features)
            deem_file = f'{deem_dir}/{cutoff}/soaps.hdf5'
            iza_file = f'{iza_dir}/{cutoff}/soaps.hdf5'

            # Assemble the train and test set SOAPs from IZA and DEEM
            soaps_train, soaps_test = load_soaps(deem_file, iza_file,
                                                 idxs_deem_train, idxs_deem_test,
                                                 idxs_iza_train, idxs_iza_test,
                                                 idxs_iza_delete=[RWY])

            # Compute kernels
            kernel_parameters = load_json(kernel_parameter_file)
            K_train, K_test, K_test_test = \
                compute_kernels(soaps_train, soaps_test, 
                                kernel_file=kernel_file, **kernel_parameters)

# Kernel Models

## Optimize KernelSVC parameters

In [ ]:
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_model_dir}/{kernel_name}/{cutoff}/structure_kernels_optimization.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = load_kernels(kernel_file)
        
        for n_cantons in (2, 4):
            
            # File to store the KSVC hyperparameters
            output_dir = f'KSVC-KPCovR/{n_cantons}-class/{kernel_name}'
            ksvc_parameter_file = f'{kernel_model_dir}/{cutoff}/{output_dir}/ksvc_parameters.json'
            ksvc_parameters = svc_parameters['kernel'].copy()
            
            for cdx, c in enumerate(C):
                
                # TODO: swap order so kernel operations aren't re-done all the time
                for n in range(0, n_subsets):
                    
                    idxs_train = idxs_train_kernel[n]
                    idxs_test = idxs_test_kernel[n]
                    
                    k_train = K_train[idxs_train, :][:, idxs_train]
                    k_test = K_test[idxs_test, :][:, idxs_train]
                    k_test_test = K_test_test[idxs_test, :][:, idxs_test]
                    
                    # Center and scale kernels
                    k_train, k_test, k_test_test = \
                        preprocess_kernels(k_train, [k_test], [k_test_test])
                    
                    y_train = cantons_train[idxs_train]
                    y_test = cantons_test[idxs_test]
                    
                    # Run KSVC                    
                    df_train, df_test, predicted_cantons_train, predicted_cantons_test = \
                        do_svc(k_train, k_test, y_train, y_test, 
                               svc_type='kernel', **ksvc_parameters, C=c, outputs=['scores'])
                    
                    class_accuracy_train[cdx, n] = train_score
                    class_accuracy_test[cdx, n] = test_score
            
            # TODO: use accuracy only on IZA structures?
            class_accuracy_train_avg = np.mean(class_accuracy_train, axis=1)
            class_accuracy_test_avg = np.mean(class_accuracy_test, axis=1)
            
            idx_C = np.argmax(class_accuracy_test_avg)
            C_opt = C[idx_C]
            ksvc_parameters.update(C=C_opt)
            save_json(ksvc_parameter_file, ksvc_parameters)    

## Optimal KernelSVC

In [ ]:
model_type = 'kernel'
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_model_dir}/{kernel_type.capitalize()}/{cutoff}/structure_kernels_optimization.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = load_kernels(kernel_file)
        
        # Since we need decision functions for all IZA structures in the train set,
        # predict all structures based on the truncated train set below
        k_train_all = K_train[:, idxs_train_all]
        k_test_all = K_test[:, idxs_train_all]

        k_train = k_train_all[idxs_train_all, :]
        k_test = K_test_all[idxs_test_all, :]
        
        k_train, k_test, k_train_all, k_test_all = \
            preprocess_kernels(k_train, K_test=[k_test, k_train_all, k_test_all])
        
        for n_cantons in (2, 4):
            
            # File to store the KSVC hyperparameters
            output_dir = f'KSVC-KPCovR/{n_cantons}-class/{kernel_name}'
            ksvc_parameter_file = f'{kernel_model_dir}/{cutoff}/{output_dir}/ksvc_parameters.json'
            
            # Files to store IZA and DEEM KSVC decision functions
            ksvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
            ksvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
                    
            # Assemble properties
            y_train = cantons_train[idxs_train_all]
            y_test = cantons_test[idxs_test_all]
            
            # KSVC
            ksvc_parameters = load_json(ksvc_parameter_file)
            df_train, df_test, train_score, test_score = \
                do_svc(k_train, k_test, y_train, y_test, svc_type='kernel',
                       outputs=['decision_functions', 'scores'])
            
            # Save decision functions for KRR test and KPCovR optimization
            split_and_save(df_train, df_test,
                           idxs_deem_train, idxs_deem_test,
                           deem_train_slice, deem_test_slice,
                           hdf5_attrs=None,
                           output=ksvc_df_deem_file, output_format='%f')
            
            split_and_save(df_train, df_test,
                           idxs_iza_train, idxs_iza_test,
                           iza_train_slice, iza_test_slice,
                           hdf5_attrs=None,
                           output=ksvc_df_iza_file, output_format='%f')

## KRR check

In [ ]:
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_model_dir}/{kernel_name}/{cutoff}/structure_kernels_optimization.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = load_kernels(kernel_file)
        
        # Since we need decision functions for all IZA structures in the train set,
        # predict all structures based on the truncated train set below
        k_train_all = K_train[:, idxs_train_all]
        k_test_all = K_test[:, idxs_train_all]

        k_train = k_train_all[idxs_train_all, :]
        k_test = K_test_all[idxs_test_all, :]

        k_train, k_test, k_train_all, k_test_all = \
            preprocess_kernels(k_train, K_test=[k_test, k_train_all, k_test_all])
        
        for n_cantons in (2, 4):
            
            # Directory to put all the output data in
            output_dir = f'KSVC-KPCovR/{n_cantons}-class/{kernel_name}'
            
            # Load decision functions
            ksvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
            ksvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
            
            df_train, df_test = load_decision_functions(ksvc_df_deem_file, ksvc_df_iza_file,
                                                        idxs_deem_train, idxs_deem_test,
                                                        idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                preprocess_decision_functions(df_train, df_test)

            # Check that KRR can reproduce the decision functions
            regression_check(K_train, K_test, df_train, df_test, regression_type='kernel')

## Optimize KPCovR parameters

In [ ]:
model_type = 'kernel'
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        kernel_name = kernel_type.capitalize()
        
        # File to store kernels for re-use
        kernel_file = f'{kernel_model_dir}/{kernel_name}/{cutoff}/structure_kernels_optimization.hdf5'
        
        # Load kernels
        K_train, K_test, K_test_test = load_kernels(kernel_file)
        
        for n_cantons in (2, 4):
            
            # Load decision functions
            ksvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
            ksvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/ksvc_structure_dfs_optimization.dat'
            
            df_train, df_test = load_decision_functions(ksvc_df_deem_file, ksvc_df_iza_file,
                                                        idxs_deem_train, idxs_deem_test,
                                                        idxs_iza_train, idxs_iza_test)
            
            # Initialize matrices of losses
            n_df = df_train.shape[1]
            lrs_train = np.zeros((alphas.size, regularizations.size, n_df))
            lks_train = np.zeros((alphas.size, regularizations.size, n_df))
            lrs_test = np.zeros((alphas.size, regularizations.size, n_df))
            lks_test = np.zeros((alphas.size, regularizations.size, n_df))
            ltrs_train = np.zeros((alphas.size, regularizations.size))
            ltks_train = np.zeros((alphas.size, regularizations.size))
            ltrs_test = np.zeros((alphas.size, regularizations.size))
            ltks_test = np.zeros((alphas.size, regularizations.size))
            
            # Directory to put all the output data in
            output_dir = f'KSVC-KPCovR/{n_cantons}-class/{kernel_name}'
            
            kpcovr_parameter_file = f'{model_dir}/{cutoff}/{output_dir}/kpcovr_parameters.json'
            
            for adx, a in enumerate(alphas):
                for rdx, r in enumerate(regularizations):
                                        
                    # TODO: swap order so kernel operations aren't re-done all the time
                    for n in range(0, n_subsets):
                        
                        # Assemble kernels
                        idxs_train = idxs_train_kernel[n]
                        k_train = K_train[idxs_train, :][:, idxs_train]

                        idxs_test = idxs_test_kernel[n]
                        k_test = K_test[idxs_test, :][:, idxs_train]
                        k_test_test = K_test_test[idxs_test, :][:, idxs_test]
                        
                        k_train, k_test, k_test_test = \
                            preprocess_kernels(k_train, K_test=[k_test], K_test_test=[k_test_test])

                        # Assemble properties
                        y_train, y_test, y_center, y_scale = \
                            preprocess_decision_functions(df_train[idxs_train], df_test[idxs_test])
                        
                        # Run KPCovR
                        kpcovr_parameters = covr_kwargs['kernel'].copy()
                        t_train, t_test, yp_train, yp_test = \
                            do_covr(k_train, k_test, y_train, y_test, 
                                    covr_type='kernel', **kpcovr_parameters,
                                    alpha=a, regularization=r)

                        # Post process the KPCovR decision functions
                        # (i.e., turn them back into canton predictions)
                        predicted_cantons_train, predicted_cantons_test = \
                            postprocess_decision_functions(yp_train, yp_test, y_center, y_scale)

                        # Collect regression and projection (KPCovR) losses for each canton
                        # TODO: losses on just IZA?
                        lrs_train, lrs_test, ltrs_train, ltrs_test = \
                            compute_regression_losses(lrs_train, lrs_test, ltrs_train, ltrs_test,
                                                      y_train, y_test, yp_train, yp_test)
                        
                        lks_train, lks_test, ltks_train, ltks_test = \
                            compute_kernel_projection_losses(lks_train, lks_test, ltks_train, ltks_test,
                                                             k_train, k_test, k_test_test, 
                                                             t_train, t_test)

                    # Average over the folds
                    lrs_train, lrs_test, lks_train, lks_test, \
                    ltrs_train, ltrs_test, ltks_train, ltks_test = \
                        average_losses(lrs_train, lrs_test, lks_train, lks_test,
                                       ltrs_train, ltrs_test, ltks_train, ltks_test)
                    
                    # TODO: save losses

            # Extract optimal hyperparameters
            ltprs_test = ltrs_test + ltps_test
            opt_alpha_idx, opt_reg_idx = np.unravel_index(np.argmin(ltprs_test, axis=None), 
                                                          ltprs_test.shape)

            opt_alpha = alphas[opt_alpha_idx]
            opt_reg = regularizations[opt_reg_idx]

            print(opt_alpha, opt_reg)

            # Save KPCovR parameters
            kpcovr_parameters = dict(n_components=n_components,
                                     regularization=opt_reg,
                                     alpha=opt_alpha)
            
            save_json(kpcovr_parameters, kpcovr_parameter_file)

In [ ]:
# DON'T MOVE THESE TO PROJECT UTILS
def compute_regression_losses(lrs_train, lrs_test, lrts_train, ltrs_test, 
                              y_train, y_test, yp_train, yp_test):
    
    lr_test = l_regr(y_test, yp_test)
    lr_train = l_regr(y_train, yp_train)
    
    # Total loss as sum of cantonwise losses
    ltr_train = np.sum(lrs_train)
    ltr_test = np.sum(lrs_test)

    return lrs_train + lr_train, lrs_test + lr_test, \
        ltrs_train + ltr_train, ltrs_test + ltr_test

def compute_kernel_projection_losses(lps_train, lps_test, ltps_train, ltps_test,
                                     k_train, k_test, k_test_test, t_train, t_test):
    
    lp_test = l_kpcovr(k_train=k_train,
                       k_test=k_test,
                       k_test_test=k_test_test,
                       t_train=t_train, t_test=t_test)

    lp_train = l_kpcovr(k_train=k_train,
                        t_train=t_train, t_test=t_test)

    # Total loss as sum of cantonwise losses
    ltp_train = np.sum(lp_train)
    ltp_test = np.sum(lp_test)
    
    return lps_train + lp_train, lps_test + lp_test,\
        ltps_train + ltp_train, ltps_test + ltp_test

def compute_linear_projection_losses(lps_train, lps_test, ltps_train, ltps_test,
                                     x_train, x_test, xr_train, xr_test):
    
    lp_train = l_proj(x_train, xr=xr_train)
    lp_test = l_proj(x_test, xr=xr_test)

    # Total loss as sum of cantonwise losses
    ltp_train = np.sum(lps_train)
    ltp_test = np.sum(lps_test)
    
    return lps_train + lp_train, lps_test + lp_test, \
        ltps_train + ltp_train, ltps_test + ltp_test

def average_losses(lrs_train, lrs_test, lks_train, lks_test,
                   ltrs_train, ltrs_test, ltks_train, ltks_test):
    
    # Average over the folds
    lrs_test = lrs_test / n_subsets
    lks_test = lks_test / n_subsets

    ltrs_test = ltrs_test / n_subsets
    ltks_test = ltks_test / n_subsets

    lrs_train = lrs_train / n_subsets
    lks_train = lks_train / n_subsets

    ltrs_train = ltrs_train / n_subsets
    ltks_train = ltks_train / n_subsets
    
    return lrs_train, lrs_test, lks_train, lks_test, \
        ltrs_train, ltrs_test, ltks_train, ltks_test

## Check for optimization problems

In [ ]:
model_type = 'kernel'
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'):
        
        # TODO: load losses
        
        for n_cantons in (2, 4):
            
            fig = plt.figure(figsize=(7.0, 7.0))
            axs_loss_train = fig.add_subplot(2, 2, 1)
            axs_loss_sum_train = fig.add_subplot(2, 2, 2)
            axs_loss_test = fig.add_subplot(2, 2, 3)
            axs_loss_sum_test = fig.add_subplot(2, 2, 4)

            # Cantonwise losses for the train set
            for i in range(0, 3):
                axs_loss_train.semilogy(alphas, lrs_train[:, opt_reg_idx, i], 'o-', label=f'l_regr_{i+1}')
                axs_loss_train.semilogy(alphas, lks_train[:, opt_reg_idx, i], 'o-', label=f'l_proj_{i+1}')
                axs_loss_train.semilogy(alphas, lrs_train[:, opt_reg_idx, i]+lks_train[:, opt_reg_idx, i], 
                         'o-', label=f'l_regr+l_proj_{i+1}')

            axs_loss_train.legend()
            axs_loss_train.set_title('Train')
            axs_loss_train.set_xlabel('alpha')
            axs_loss_train.set_ylabel('loss')

            # Sum of projection and regression loss over all cantons for the train set
            axs_loss_sum_train.semilogy(alphas, ltrs_train[:, opt_reg_idx], 'o-', label='l_regr')
            axs_loss_sum_train.semilogy(alphas, ltks_train[:, opt_reg_idx], 'o-', label='l_proj')
            axs_loss_sum_train.semilogy(alphas, ltrs_train[:, opt_reg_idx]+ltks_train[:, opt_reg_idx], 
                                    'o-', label='l_regr+l_proj')

            axs_loss_sum_train.legend()
            axs_loss_sum_train.set_title('Train')
            axs_loss_sum_train.set_xlabel('alpha')
            axs_loss_sum_train.set_ylabel('loss')

            # Cantonwise losses for the test set
            for i in range(0, 3):
                axs_loss_test.semilogy(alphas, lrs_test[:, opt_reg_idx, i], 'o-', label=f'l_regr_{i+1}')
                axs_loss_test.semilogy(alphas, lks_test[:, opt_reg_idx, i], 'o-', label=f'l_proj_{i+1}')
                axs_loss_test.semilogy(alphas, lrs_test[:, opt_reg_idx, i]+lks_test[:, opt_reg_idx, i], 
                         'o-', label=f'l_regr+l_proj_{i+1}')

            axs_loss_test.legend()
            axs_loss_test.set_title('Test')
            axs_loss_test.set_xlabel('alpha')
            axs_loss_test.set_ylabel('loss')

            # Sum of projection and regression loss over all cantons for the test set
            axs_loss_sum_test.semilogy(alphas, ltrs_test[:, opt_reg_idx], 'o-', label='l_regr')
            axs_loss_sum_test.semilogy(alphas, ltks_test[:, opt_reg_idx], 'o-', label='l_proj')
            axs_loss_sum_test.semilogy(alphas, ltrs_test[:, opt_reg_idx]+ltks_test[:, opt_reg_idx], 'o-', label='l_regr+l_proj')

            axs_loss_sum_test.legend()
            axs_loss_sum_test.set_title('Test')
            axs_loss_sum_test.set_xlabel('alpha')
            axs_loss_sum_test.set_ylabel('loss')

            plt.show()

# Linear

In [ ]:
n_species = 2
n_species_pairs = n_species * (n_species + 1) // 2
n_features = n_species_pairs * soap_hyperparameters['max_radial']**2 \
    * (soap_hyperparameters['max_angular'] + 1)
feature_groups = extract_species_pair_groups(n_features, n_species)

## Optimize LinearSVC parameters

In [ ]:
for cutoff in cutoffs:
    for spectrum_type in ('power', 'radial'):
        
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        # Assemble the train and test set SOAPs from IZA and DEEM
        soaps_train, soaps_test = load_soaps(deem_file, iza_file,
                                             idxs_deem_train, idxs_deem_test,
                                             idxs_iza_train, idxs_iza_test,
                                             idxs_iza_delete=[RWY])
        
        for species_pairing, feature_idxs in zip(('OO', 'OSi', 'SiSi'), feature_groups):
            for n_cantons in (2, 4):
                # TODO: MOVE KERNELS BEFORE RUNNING!

                # Directory to put all the output data in
                output_dir = f'LSVC-PCovR/{n_cantons}-class/{spectrum_type}/{species_pairing}'
                lsvc_parameter_file = f'{linear_model_dir}/{cutoff}/{output_dir}/lsvc_parameters.json'
                lsvc_parameters = svc_parameters['linear'].copy()

                if not os.path.exists(output_dir):
                    os.makedirs(output_dir)

                for cdx, c in enumerate(C):

                    # TODO: swap order so kernel operations aren't re-done all the time
                    for n in range(0, n_subsets):

                        idxs_train = idxs_train_kernel[n]
                        idxs_test = idxs_test_kernel[n]

                        x_train = soaps_train[idxs_train, feature_idxs]
                        x_test = soaps_test[idxs_test, feature_idxs]
                        
                        # Scale the SOAPs so they are of a 'usable' magnitude for the SVC
                        x_train, x_test = preprocess_soaps(x_train, x_test)

                        y_train = cantons_train[idxs_train]
                        y_test = cantons_test[idxs_test]

                        # Run LSVC
                        train_score, test_score = \
                            do_svc(x_train, x_test, y_train, y_test, 
                                   svc_type='linear', **lsvc_parameters, C=c, outputs=['scores'])

                        class_accuracy_train[cdx, n] = train_score
                        class_accuracy_test[cdx, n] = test_score

                # TODO: use accuracy only on IZA structures?
                # TODO: put this in a function? it's same as the kernel version
                class_accuracy_train_avg = np.mean(class_accuracy_train, axis=1)
                class_accuracy_test_avg = np.mean(class_accuracy_test, axis=1)

                idx_C = np.argmax(class_accuracy_test_avg)
                C_opt = C[idx_C]
                ksvc_parameters.update(C=C_opt)
                save_json(lsvc_parameter_file, lsvc_parameters)    

## Optimal LinearSVC

In [ ]:
for cutoff in cutoffs:
    for spectrum_type in ('power', 'radial'):
        
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        
        # Assemble the train and test set SOAPs from IZA and DEEM
        soaps_train, soaps_test = load_soaps(deem_file, iza_file,
                                             idxs_deem_train, idxs_deem_test,
                                             idxs_iza_train, idxs_iza_test,
                                             idxs_iza_delete=[RWY])
        
        for species_pairing, feature_idxs in zip(('OO', 'OSi', 'SiSi'), feature_groups):
                        
            for n_cantons in (2, 4):

                # Files to store the LSVC hyperparameters
                output_dir = f'LSVC-PCovR/{n_cantons}-class/{model_name}'
                lsvc_parameter_file = f'{linear_model_dir}/{cutoff}/{output_dir}/lsvc_parameters.json'

                # Files to store IZA and DEEM LSVC decision functions
                lsvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'
                lsvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'

                # Assemble properties
                y_train = cantons_train[idxs_train_all]
                y_test = cantons_test[idxs_test_all]
                
                x_train = soaps_train[idxs_train_all, feature_idxs]
                x_test = soaps_test[idxs_train_all, feature_idxs]
                
                x_train, x_test = preprocess_soaps(x_train, x_test)

                # LSVC
                lsvc_parameters = load_json(lsvc_parameter_file)
                df_train, df_test, train_score, test_score = \
                    do_svc(soaps_train, soaps_test, y_train, y_test, svc_type='linear',
                           outputs=['decision_functions', 'scores'])

                # Save decision functions for LR test and PCovR optimization
                split_and_save(df_train, df_test,
                               idxs_deem_train, idxs_deem_test,
                               deem_train_slice, deem_test_slice,
                               hdf5_attrs=None,
                               output=lsvc_df_deem_file, output_format='%f')

                split_and_save(df_train, df_test,
                               idxs_iza_train, idxs_iza_test,
                               iza_train_slice, iza_test_slice,
                               hdf5_attrs=None,
                               output=lsvc_df_iza_file, output_format='%f')

## LR check

# TODO: START HERE

In [ ]:
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'): # TODO: change to drop-out models
        model_name = 'placeholder'
        
        # TODO: load SOAPs
        # TODO: preprocess SOAPs
        
        for n_cantons in (2, 4):
            
            # Directory to put all the output data in
            output_dir = f'LSVC-PCovR/{n_cantons}-class/{model_name}'
            
            # Files to store IZA and DEEM KSVC decision functions
            lsvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'
            lsvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'
            
            df_train, df_test = load_data(lsvc_df_deem_file, lsvc_df_iza_file,
                                          idxs_deem_train, idxs_deem_test,
                                          idxs_iza_train, idxs_iza_test)
            
            # Center and scale the decision functions
            df_train, df_test, df_center, df_scale = \
                preprocess_decision_functions(df_train, df_test)

            # Check that KRR can reproduce the decision functions
            regression_check(soaps_train, soaps_test, df_train, df_test, regression_type='linear')

## Optimize PCovR parameters

In [ ]:
model_type = 'kernel'
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'): # TODO: change to drop-out models
        model_name = 'placeholder'
        
        # TODO: load SOAPs
        
        for n_cantons in (2, 4):
            
            # Load decision functions
            lsvc_df_deem_file = f'{deem_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'
            lsvc_df_iza_file = f'{iza_dir}/{cutoff}/{output_dir}/lsvc_structure_dfs_optimization.dat'
            
            df_train, df_test = load_decision_functions(lsvc_df_deem_file, lsvc_df_iza_file,
                                                        idxs_deem_train, idxs_deem_test,
                                                        idxs_iza_train, idxs_iza_test)
            
            # Initialize matrices of losses
            n_df = df_train.shape[1]
            lrs_train = np.zeros((alphas.size, regularizations.size, n_df))
            lks_train = np.zeros((alphas.size, regularizations.size, n_df))
            lrs_test = np.zeros((alphas.size, regularizations.size, n_df))
            lks_test = np.zeros((alphas.size, regularizations.size, n_df))
            ltrs_train = np.zeros((alphas.size, regularizations.size))
            ltks_train = np.zeros((alphas.size, regularizations.size))
            ltrs_test = np.zeros((alphas.size, regularizations.size))
            ltks_test = np.zeros((alphas.size, regularizations.size))
            
            # Directory to put all the output data in
            output_dir = f'SVC-PCovR/{n_cantons}-class/{model_name}'
            
            pcovr_parameter_file = f'{model_dir}/{cutoff}/{output_dir}/pcovr_parameters.json'
            
            for adx, a in enumerate(alphas):
                for rdx, r in enumerate(regularizations):
                    
                    # TODO: swap order so kernel operations aren't re-done all the time
                    for n in range(0, n_subsets):
                        
                        # TODO: slice SOAPs
                        # e.g., x_train = soaps_train[slice]
                        # TODO: preprocess SOAPs

                        # Assemble properties
                        y_train, y_test, y_center, y_scale = \
                            preprocess_decision_functions(df_train[idxs_train], df_test[idxs_test])

                        # Run KPCovR
                        pcovr_parameters = covr_kwargs['linear'].copy()
                        t_train, t_test, yp_train, yp_test = \
                            do_covr(x_train, x_test, y_train, y_test, 
                                    covr_type='linear', **pcovr_parameters,
                                    alpha=a, regularization=r)

                        # Post process the KPCovR decision functions
                        # (i.e., turn them back into canton predictions)
                        predicted_cantons_train, predicted_cantons_test = \
                            postprocess_decision_functions(yp_train, yp_test, y_center, y_scale)

                        # Collect regression and projection (KPCovR) losses for each canton
                        lrs_train, lrs_test, ltrs_train, ltrs_test = \
                            compute_regression_losses(lrs_train, lrs_test, ltrs_train, ltrs_test,
                                                      y_train, y_test, yp_train, yp_test)
                        
                        lks_train, lks_test, ltks_train, ltks_test = \
                            compute_linear_projection_losses(lks_train, lks_test, ltks_train, ltks_test,
                                                             k_train, k_test, k_test_test, 
                                                             t_train, t_test)

                    # Average over the folds
                    lrs_train, lrs_test, lks_train, lks_test, \
                    ltrs_train, ltrs_test, ltks_train, ltks_test = \
                        average_losses(lrs_train, lrs_test, lks_train, lks_test,
                                       ltrs_train, ltrs_test, ltks_train, ltks_test)
                    
                    # TODO: save losses

            # Extract optimal hyperparameters
            ltprs_test = ltrs_test + ltps_test
            opt_alpha_idx, opt_reg_idx = np.unravel_index(np.argmin(ltprs_test, axis=None), 
                                                          ltprs_test.shape)

            opt_alpha = alphas[opt_alpha_idx]
            opt_reg = regularizations[opt_reg_idx]

            print(opt_alpha, opt_reg)

            # Save PCovR parameters
            pcovr_parameters = dict(n_components=n_components,
                                    regularization=opt_reg,
                                    alpha=opt_alpha)
            
            save_json(pcovr_parameters, pcovr_parameter_file)

## Check for optimization problems

In [ ]:
for cutoff in cutoffs:
    for kernel_type in ('linear', 'gaussian'): # TODO: change to drop-out models
        
        # TODO: load losses
        
        for n_cantons in (2, 4):
            
            fig = plt.figure(figsize=(7.0, 7.0))
            axs_loss_train = fig.add_subplot(2, 2, 1)
            axs_loss_sum_train = fig.add_subplot(2, 2, 2)
            axs_loss_test = fig.add_subplot(2, 2, 3)
            axs_loss_sum_test = fig.add_subplot(2, 2, 4)

            # Cantonwise losses for the train set
            for i in range(0, 3):
                axs_loss_train.semilogy(alphas, lrs_train[:, opt_reg_idx, i], 'o-', label=f'l_regr_{i+1}')
                axs_loss_train.semilogy(alphas, lks_train[:, opt_reg_idx, i], 'o-', label=f'l_proj_{i+1}')
                axs_loss_train.semilogy(alphas, lrs_train[:, opt_reg_idx, i]+lks_train[:, opt_reg_idx, i], 
                         'o-', label=f'l_regr+l_proj_{i+1}')

            axs_loss_train.legend()
            axs_loss_train.set_title('Train')
            axs_loss_train.set_xlabel('alpha')
            axs_loss_train.set_ylabel('loss')

            # Sum of projection and regression loss over all cantons for the train set
            axs_loss_sum_train.semilogy(alphas, ltrs_train[:, opt_reg_idx], 'o-', label='l_regr')
            axs_loss_sum_train.semilogy(alphas, ltks_train[:, opt_reg_idx], 'o-', label='l_proj')
            axs_loss_sum_train.semilogy(alphas, ltrs_train[:, opt_reg_idx]+ltks_train[:, opt_reg_idx], 
                                    'o-', label='l_regr+l_proj')

            axs_loss_sum_train.legend()
            axs_loss_sum_train.set_title('Train')
            axs_loss_sum_train.set_xlabel('alpha')
            axs_loss_sum_train.set_ylabel('loss')

            # Cantonwise losses for the test set
            for i in range(0, 3):
                axs_loss_test.semilogy(alphas, lrs_test[:, opt_reg_idx, i], 'o-', label=f'l_regr_{i+1}')
                axs_loss_test.semilogy(alphas, lks_test[:, opt_reg_idx, i], 'o-', label=f'l_proj_{i+1}')
                axs_loss_test.semilogy(alphas, lrs_test[:, opt_reg_idx, i]+lks_test[:, opt_reg_idx, i], 
                         'o-', label=f'l_regr+l_proj_{i+1}')

            axs_loss_test.legend()
            axs_loss_test.set_title('Test')
            axs_loss_test.set_xlabel('alpha')
            axs_loss_test.set_ylabel('loss')

            # Sum of projection and regression loss over all cantons for the test set
            axs_loss_sum_test.semilogy(alphas, ltrs_test[:, opt_reg_idx], 'o-', label='l_regr')
            axs_loss_sum_test.semilogy(alphas, ltks_test[:, opt_reg_idx], 'o-', label='l_proj')
            axs_loss_sum_test.semilogy(alphas, ltrs_test[:, opt_reg_idx]+ltks_test[:, opt_reg_idx], 'o-', label='l_regr+l_proj')

            axs_loss_sum_test.legend()
            axs_loss_sum_test.set_title('Test')
            axs_loss_sum_test.set_xlabel('alpha')
            axs_loss_sum_test.set_ylabel('loss')

            plt.show()

# Logistic Regression